In [25]:
!pip install pymysql
!pip install yahooquery


  Obtaining dependency information for yahooquery from https://files.pythonhosted.org/packages/d1/a9/9a06f31cc068c7997b63a358f94ed433afb28599ef63bdcd5333db4a19d8/yahooquery-2.3.7-py3-none-any.whl.metadata
  Obtaining dependency information for requests-futures<2.0.0,>=1.0.1 from https://files.pythonhosted.org/packages/af/ca/ca664ed374fab67535482532c3c05bb5dbe5850e7dff2491eb827c318e48/requests_futures-1.0.1-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/52.7 kB ? eta -:--:--
   ------- -------------------------------- 10.2/52.7 kB ? eta -:--:--
   ---------------------------------------- 52.7/52.7 kB 904.4 kB/s eta 0:00:00


In [31]:
import pymysql
from datetime import datetime
from yahooquery import Ticker

# Replace "demo" with your Alpha Vantage API key
api_key = 'your_alpha_vantage_api_key'
symbols = ['IBM', 'AAPL', 'GOOGL', 'MSFT']

# Function to fetch real-time stock data from Yahoo Finance API
def get_stock_data(symbol):
    try:
        ticker = Ticker(symbol)
        data = ticker.history(period='1d')
        
        if 'close' in data and not data['close'].empty:
            latest_data = data['close'].iloc[-1]
            open_price = data['open'].iloc[-1]
            high_price = data['high'].iloc[-1]
            low_price = data['low'].iloc[-1]
            volume = data['volume'].iloc[-1]
            
            return {
                'symbol': symbol,
                'price': latest_data,
                'open': open_price,
                'high': high_price,
                'low': low_price,
                'volume': volume,
                'timestamp': datetime.now()
            }
        else:
            return None
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

# Function to insert stock data into the MySQL database
def insert_stock_data(data, connection):
    try:
        with connection.cursor() as cursor:
            # Insert data into the 'stock_data' table
            sql = "INSERT INTO stock_data (symbol, price, open, high, low, volume, timestamp) VALUES (%s, %s, %s, %s, %s, %s, %s)"
            cursor.execute(sql, (data['symbol'], data['price'], data['open'], data['high'], data['low'], data['volume'], data['timestamp']))
        # Commit the changes to the database
        connection.commit()
        print(f"Data for {data['symbol']} inserted successfully.")
    except pymysql.Error as e:
        # Handle any database-related errors
        print(f"Error: {e}")

# Replace these values with your actual MySQL credentials
host = 'localhost'
user = 'root'
password = 'root'
database = 'sri'

try:
    # Attempt to establish a connection to the database
    connection = pymysql.connect(host=host, user=user, password=password, database=database)
    
    # Fetch real-time data for each stock symbol using Yahoo Finance API
    for symbol in symbols:
        stock_data = get_stock_data(symbol)
        
        if stock_data is not None:
            insert_stock_data(stock_data, connection)
        else:
            print(f"Failed to fetch data for {symbol}.")

except pymysql.Error as e:
    # If there is an error connecting to the database, print an error message
    print(f"Error: {e}")

finally:
    # Close the connection in the 'finally' block to ensure it's closed
    if connection:
        connection.close()
        print("Connection closed.")


Data for IBM inserted successfully.
Data for AAPL inserted successfully.
Data for GOOGL inserted successfully.
Data for MSFT inserted successfully.
Connection closed.


In [33]:
import logging

logging.basicConfig(filename='stock_data_fetch.log', level=logging.INFO)

# ... (previous code)

try:
    logging.info("Fetching real-time data from Alpha Vantage API...")
    stock_price = get_stock_data(symbol, api_key)
    
    if stock_price is not None:
        logging.info(f"Stock Price for {symbol}: {stock_price}")
        insert_stock_data(symbol, stock_price, connection)
    else:
        logging.warning(f"Failed to fetch data for {symbol}.")

except Exception as e:
    logging.error(f"An error occurred: {e}")

# ... (remaining code)
